In [1]:
import torch

class SimpleSCMPrior:
    """
    단순한 구조적 인과 모델(SCM)을 사용하여 합성 데이터를 생성하는 클래스.
    생성 시 n_features와 n_classes를 설정받습니다.
    """
    def __init__(self, n_features=100, n_classes=2):
        """
        클래스 생성자 (Initializer)
        생성 시 전달받은 특성 및 클래스 개수를 인스턴스 변수로 저장합니다.
        """
        self.n_features = n_features
        self.n_classes = n_classes

    def sample(self, batch_size=64, n_samples=100):
        """
        SCM으로부터 하나의 배치 데이터를 샘플링합니다.
        """
        # 1. 특성(X) 및 잠재 변수(logits) 생성
        # __init__에서 저장한 self.n_features를 사용합니다.
        X = torch.randn(batch_size, n_samples, self.n_features)
        causal_weights = torch.randn(batch_size, self.n_features, 1)
        logits = X @ causal_weights

        # 2. 안정적인 방식으로 임계값(Thresholds) 계산
        min_vals, _ = torch.min(logits, dim=1, keepdim=True)
        max_vals, _ = torch.max(logits, dim=1, keepdim=True)
        
        # __init__에서 저장한 self.n_classes를 사용합니다.
        ratios = torch.linspace(0, 1, self.n_classes + 1, device=logits.device)[1:-1]
        ratios = ratios.view(1, -1, 1)
        thresholds = min_vals + (max_vals - min_vals) * ratios
        
        # 3. 로짓과 임계값 비교를 통한 레이블(y) 생성
        y = torch.sum(logits > thresholds.transpose(1, 2), dim=2)

        return X.float(), y.long()

# 사용 예시
prior = SimpleSCMPrior()
X_batch, y_batch = prior.sample()
print(X_batch.shape, y_batch.shape) # torch.Size() torch.Size()

torch.Size([64, 100, 100]) torch.Size([64, 100])


In [2]:
import torch.nn as nn
import torch.nn.functional as F

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src, src_mask=None):
        # Multi-head Self-attention
        src2, _ = self.self_attn(src, src, src, attn_mask=src_mask)
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        
        # Feed-forward Network
        src2 = self.linear2(self.dropout(F.relu(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src

class TransformerEncoder(nn.Module):
    def __init__(self, encoder_layer, num_layers):
        super().__init__()
        self.layers = nn.ModuleList([encoder_layer for _ in range(num_layers)])
        self.num_layers = num_layers

    def forward(self, src, mask=None):
        output = src
        for mod in self.layers:
            output = mod(output, src_mask=mask)
        return output

In [3]:
class PFN(nn.Module):
    def __init__(self, d_model=256, nhead=4, num_encoder_layers=6, n_features=10, n_classes=2):
        super().__init__()
        self.d_model = d_model
        
        # 입력 임베딩 레이어
        self.feature_embedding = nn.Linear(n_features, d_model)
        self.label_embedding = nn.Embedding(n_classes, d_model)
        
        # 트랜스포머 인코더
        encoder_layer = TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_encoder_layers)
        
        # 출력(예측) 헤드
        self.output_head = nn.Linear(d_model, n_classes)

    def _generate_attention_mask(self, n_train, n_test, device):
        """테스트 포인트가 서로 attend하지 못하도록 마스크 생성"""
        total_len = n_train + n_test
        mask = torch.zeros(total_len, total_len, device=device).bool()
        # 테스트 -> 테스트 영역을 True로 설정하여 어텐션 차단
        mask[n_train:, n_train:] = True
        # 대각선은 False로 유지 (자기 자신은 attend 가능)
        mask.fill_diagonal_(False)
        return mask

    def forward(self, train_x, train_y, test_x):
        # train_x: (batch, n_train, n_features)
        # train_y: (batch, n_train)
        # test_x: (batch, n_test, n_features)
        
        batch_size, n_train, _ = train_x.shape
        _, n_test, _ = test_x.shape

        # 1. 토큰화 (Embedding)
        # 훈련 데이터: 특성과 레이블 임베딩을 합침
        train_x_emb = self.feature_embedding(train_x)
        train_y_emb = self.label_embedding(train_y)
        train_tokens = train_x_emb + train_y_emb

        # 테스트 데이터: 특성만 임베딩 (레이블은 예측 대상)
        test_tokens = self.feature_embedding(test_x)

        # 2. 시퀀스 결합
        # (batch, n_train + n_test, d_model)
        full_sequence = torch.cat([train_tokens, test_tokens], dim=1)

        # 3. 어텐션 마스크 생성
        attn_mask = self._generate_attention_mask(n_train, n_test, train_x.device)

        # 4. 트랜스포머 순전파
        transformer_output = self.transformer_encoder(full_sequence, mask=attn_mask)

        # 5. 예측 헤드
        # 테스트 데이터에 해당하는 출력만 사용
        test_output_embeddings = transformer_output[:, n_train:, :]
        logits = self.output_head(test_output_embeddings) # (batch, n_test, n_classes)
        
        return F.log_softmax(logits, dim=-1)

In [4]:
import torch.optim as optim

# 하이퍼파라미터
N_FEATURES = 10
N_CLASSES = 2
D_MODEL = 128
N_HEAD = 4
NUM_LAYERS = 4
BATCH_SIZE = 32
N_SAMPLES = 1024 # 사전 분포에서 샘플링할 데이터셋의 크기
TRAIN_RATIO = 0.5
LEARNING_RATE = 1e-4
EPOCHS = 10000

# 모델, 사전 분포, 옵티마이저 초기화
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
prior = SimpleSCMPrior(n_features=N_FEATURES, n_classes=N_CLASSES)
model = PFN(d_model=D_MODEL, nhead=N_HEAD, num_encoder_layers=NUM_LAYERS, 
            n_features=N_FEATURES, n_classes=N_CLASSES).to(device)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
loss_fn = nn.NLLLoss()

# 훈련 루프
for epoch in range(EPOCHS):
    model.train()
    
    # 1. 사전 분포에서 데이터셋 배치 샘플링
    X, y = prior.sample(batch_size=BATCH_SIZE, n_samples=N_SAMPLES)
    X, y = X.to(device), y.to(device)
    
    # 2. 훈련/테스트 분할
    n_train = int(N_SAMPLES * TRAIN_RATIO)
    train_x, train_y = X[:, :n_train], y[:, :n_train]
    test_x, test_y = X[:, n_train:], y[:, n_train:]
    
    # 3. 순전파 및 손실 계산
    optimizer.zero_grad()
    predicted_log_probs = model(train_x, train_y, test_x) # (batch, n_test, n_classes)
    
    # 손실 계산을 위해 차원 재정렬
    # (batch * n_test, n_classes)와 (batch * n_test)
    loss = loss_fn(predicted_log_probs.reshape(-1, N_CLASSES), test_y.reshape(-1))
    
    # 4. 역전파 및 파라미터 업데이트
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# 모델 저장
torch.save(model.state_dict(), "simple_pfn_model.pt")

Epoch 0, Loss: 0.8416905403137207
Epoch 100, Loss: 0.6191109418869019
Epoch 200, Loss: 0.20562173426151276
Epoch 300, Loss: 0.15846505761146545
Epoch 400, Loss: 0.15683847665786743
Epoch 500, Loss: 0.1532370001077652
Epoch 600, Loss: 0.14319001138210297
Epoch 700, Loss: 0.1472017467021942
Epoch 800, Loss: 0.14867673814296722
Epoch 900, Loss: 0.13636593520641327
Epoch 1000, Loss: 0.13314341008663177
Epoch 1100, Loss: 0.13291247189044952
Epoch 1200, Loss: 0.13408222794532776
Epoch 1300, Loss: 0.12257137894630432
Epoch 1400, Loss: 0.12016956508159637
Epoch 1500, Loss: 0.11561991274356842
Epoch 1600, Loss: 0.11991330981254578
Epoch 1700, Loss: 0.11594480276107788
Epoch 1800, Loss: 0.11133155971765518
Epoch 1900, Loss: 0.10503566265106201
Epoch 2000, Loss: 0.10347411036491394
Epoch 2100, Loss: 0.09855938702821732
Epoch 2200, Loss: 0.10291358083486557
Epoch 2300, Loss: 0.10001041740179062
Epoch 2400, Loss: 0.10080769658088684
Epoch 2500, Loss: 0.09456207603216171
Epoch 2600, Loss: 0.09484438